In [28]:
import time
# start_time = time.time()

import numpy as np

# from utils import plot_pr
# from utils import load_sanders_data
# from utils import tweak_labels

In [30]:
import pandas as pd

url = "https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv"
temp = "C:/z_temp/data/sanders-twitter-0.2/full-corpus.csv"
sanders_data = pd.read_csv(temp, parse_dates=[3])

sanders_data.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,2011-10-18 21:53:25,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,2011-10-18 21:09:33,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,2011-10-18 21:02:20,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,2011-10-18 20:40:10,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,2011-10-18 20:34:00,I just realized that the reason I got into twi...


In [34]:
X_orig, Y_orig = sanders_data.TweetText, sanders_data.Sentiment
classes = np.unique(Y_orig)

In [35]:
for c in classes:
    print "#%s: %i" % (c, sum(Y_orig == c))

#irrelevant: 1689
#negative: 572
#neutral: 2333
#positive: 519


## Solving an easy problem first

In [39]:
def tweak_labels(Y, pos_sent_list):
    pos = Y == pos_sent_list[0]
    for sent_label in pos_sent_list[1:]:
        pos |= Y == sent_label

    Y = np.zeros(Y.shape[0])
    Y[pos] = 1
    Y = Y.astype(int)

    return Y

In [67]:
pos_neg = np.logical_or(Y_orig == "positive", Y_orig == "negative")
X = X_orig[pos_neg]
X = np.array(X.tolist())
Y = Y_orig[pos_neg]
Y = tweak_labels(Y, ["positive"])

C:\Users\rstancut\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [69]:
def create_ngram_model():
    tfidf_ngrams = TfidfVectorizer(ngram_range=(1, 3), analyzer="word", binary=False)
    clf = MultinomialNB()
    pipeline = Pipeline([('vect', tfidf_ngrams), ('clf', clf)])
    return pipeline

In [70]:
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.cross_validation import ShuffleSplit

In [71]:
def train_model(clf_factory, X, Y, name="NB ngram", plot=False):
    cv = ShuffleSplit(n=len(X), n_iter=10, test_size=0.3, random_state=0)

    train_errors = []
    test_errors = []

    scores = []
    pr_scores = []
    precisions, recalls, thresholds = [], [], []

    for train, test in cv:
        X_train, y_train = X[train], Y[train]
        X_test, y_test = X[test], Y[test]

        clf = clf_factory()
        clf.fit(X_train, y_train)

        train_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)

        train_errors.append(1 - train_score)
        test_errors.append(1 - test_score)

        scores.append(test_score)
        proba = clf.predict_proba(X_test)

        fpr, tpr, roc_thresholds = roc_curve(y_test, proba[:, 1])
        precision, recall, pr_thresholds = precision_recall_curve(
            y_test, proba[:, 1])

        pr_scores.append(auc(recall, precision))
        precisions.append(precision)
        recalls.append(recall)
        thresholds.append(pr_thresholds)

    scores_to_sort = pr_scores
    median = np.argsort(scores_to_sort)[len(scores_to_sort) / 2]

    if plot:
#         plot_pr(pr_scores[median], name, "01", precisions[median],
#                 recalls[median], label=name)

        summary = (np.mean(scores), np.std(scores),
                   np.mean(pr_scores), np.std(pr_scores))
        print "%.3f\t%.3f\t%.3f\t%.3f\t" % summary

    return np.mean(train_errors), np.mean(test_errors)

In [72]:
train_model(create_ngram_model, X, Y, name="pos vs neg", plot=True)

C:\Users\rstancut\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\metrics\ranking.py:538: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


0.998	0.001	nan	nan	


(0.0017038007863696247, 0.0021341463414634165)

In [66]:
np.array(X)

array([ 'Now all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is',
       '@Apple will be adding more carrier support to the iPhone 4S (just announced)',
       "Hilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY",
       ...,
       '@KushKitty420 you, #Facebook #twitter and the #tv are all #distractions whenever I wanna study ;(',
       'RT @VanillaLatte_xD: #omfg .. #Twitter , Why fxck up ? I wanna see my Retweets "/',
       'RT @nicolebanyots: #twitter UH, SHOW ME MY RETWEETS, OR ILL GO BACK TO FACEBOOK. K? K.'], 
      dtype='|S148')

In [74]:
cv

NameError: name 'cv' is not defined